<font size="8"> **Adding environmental data from available observations to unique background points** </font>  
In this notebook, we will extract environmental data from sea ice and sea surface temperature observations using the coordinates of the reported crabeater sightings.

# Setting working directory
In order to ensure these notebooks work correctly, we will set the working directory. We assume that you have saved a copy of this repository in your home directory (represented by `~` in the code chunk below). If you have saved this repository elsewhere in your machine, you need to ensure you update this line with the correct filepath where you saved these notebooks.

In [1]:
import os
os.chdir(os.path.expanduser('~/Chapter2_Crabeaters/Scripts'))

# Loading other relevant libraries

In [2]:
from dask.distributed import Client
from glob import glob
#Accessing model data
import cosima_cookbook as cc
#Useful functions
import UsefulFunctions as uf
#Dealing with data
import xarray as xr
import pandas as pd
import numpy as np
#Data visualisation
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Paralellising work 

In [3]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/43559/status,
Dashboard: /proxy/43559/status,Workers: 4
Total threads: 12,Total memory: 48.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46827,Workers: 4
Dashboard: /proxy/43559/status,Total threads: 12
Started: Just now,Total memory: 48.00 GiB
Comm: tcp://127.0.0.1:32981,Total threads: 3
Dashboard: /proxy/37101/status,Memory: 12.00 GiB
Nanny: tcp://127.0.0.1:43163,


# Adding values for dynamic variables
Given the amount of crabeater seal observations and the time period covered by this dataset, the extraction of these values may take some time. It is recommended to save the data frame after every time a new variable is extracted. This way we can avoid losing data.

In [54]:
#Loading dataset as pandas data frame
crabeaters = pd.read_csv('../Biological_Data/BG_points/unique_background_20x_obs_grid.csv')

#Ensuring date column is formatted correctly (year-month)
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)

#Checking results
crabeaters

,date,year,month,longitude,latitude,xt_ocean,yt_ocean,season_year,life_stage,decade,sector,zone,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,1999-12,1999,12,75.05,-69.25,75.05,-69.239,summer,weaning,1990,East Indian,Antarctic,0,89.812,504.080,140.256,777.867
1,1999-12,1999,12,75.85,-69.25,75.85,-69.239,summer,weaning,1990,Central Indian,Antarctic,0,89.953,506.158,143.956,503.000
2,1999-12,1999,12,76.05,-69.25,76.05,-69.239,summer,weaning,1990,East Indian,Antarctic,0,89.949,507.046,145.997,522.900
3,1999-12,1999,12,77.25,-69.25,77.25,-69.239,summer,weaning,1990,Central Indian,Antarctic,0,NaN,509.341,166.208,NaN
4,1999-12,1999,12,73.95,-69.15,73.95,-69.155,summer,weaning,1990,Central Indian,Antarctic,0,NaN,494.995,130.925,599.817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35561,1999-12,1999,12,69.25,-59.45,69.25,-59.442,summer,weaning,1990,Central Indian,Antarctic,0,89.833,556.221,688.617,4624.497
35562,1999-12,1999,12,71.95,-59.45,71.95,-59.442,summer,weaning,1990,East Indian,Antarctic,0,89.653,536.648,699.380,4429.308
35563,1987-11,1987,11,70.85,-59.35,70.85,-59.340,autumn,weaning,1980,Central Indian,Antarctic,0,89.980,551.535,703.061,4767.584
35564,1998-11,1998,11,71.55,-59.35,71.55,-59.340,autumn,weaning,1990,Central Indian,Antarctic,0,89.930,548.571,707.330,4608.486


## Loading environmental data from observations

In [72]:
#Creating dictionary with useful information
varDict = {'var_original': 'dist_km',
           #Folder containing obs
           'obs_main': '/g/data/v45/la6889/Chapter2_Crabeaters/SeaIceObs/Distance_Edge/*.nc',
           #Name to store variable in final data frame
           'var_short': 'dist_ice_edge_km',
           #Output folder
           'base_out': '../Environmental_Data/Env_obs'}

In [73]:
#Getting list of all obs in folder
files_var = sorted(glob(varDict['obs_main']))
var = varDict['var_original']

#Loading all data into single dataset
var_df = xr.open_mfdataset(files_var)[var]
var_df.name = varDict['var_short']
if 'xt_ocean' not in var_df.coords:
    var_df = var_df.rename({'lon': 'xt_ocean', 'lat': 'yt_ocean'})

#Checking results
var_df

<xarray.DataArray 'dist_ice_edge_km' (time: 494, yt_ocean: 740, xt_ocean: 3600)>
dask.array<concatenate, shape=(494, 740, 3600), dtype=float64, chunksize=(1, 740, 3600), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 1978-11-15 1978-12-15 ... 2019-12-15
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... -45.18 -45.11 -45.04
  * xt_ocean  (xt_ocean) float64 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
Attributes:
    units:      km
    long_name:  distance to nearest neighbour

## Extracting environmental data
We will use the `latitude` and `longitude` columns together with the `event_date` column from the crabeater seal observations to find the corresponding grid cell in the model outputs and extract the value of the environmental factor of our interest.

In [74]:
#Getting coordinates from the crabeater data
lat = xr.DataArray(crabeaters.latitude)
lon = xr.DataArray(crabeaters.longitude)
#Getting data of observation from the crabeater data
time = xr.DataArray(crabeaters.apply(lambda x: pd.to_datetime(f'{x.date}-16'), axis = 1))

## Extracting data

In [75]:
#Extracting data
var_sub = var_df.sel(time = time, yt_ocean = lat, xt_ocean = lon, method = 'nearest')

#Transforming to data frame
var_pd = var_sub.to_dataframe().sort_values(['time', 'xt_ocean', 'yt_ocean'])
#Adding year and month
var_pd['year'] = var_pd.time.dt.year
var_pd['month'] = var_pd.time.dt.month
#Removing time column that is no longer needed
var_pd.drop(columns = 'time', inplace = True)
#Finding name of columns to round up
round_cols = [i for i in var_pd.columns if 'ocean' in i]
#Rounding coordinate values prior to merging
var_pd = var_pd.round({round_cols[0]: 3, round_cols[1]: 3})
#Getting column names for merging
cols = var_pd.drop(columns = varDict['var_short']).columns.tolist()

#Checking results
print(cols); var_pd

['yt_ocean', 'xt_ocean', 'year', 'month']


,yt_ocean,xt_ocean,dist_ice_edge_km,year,month
dim_0,,,,,
29935,-64.070,70.35,206.568023,1981,12
2585,-66.156,71.95,316.788571,1981,12
27404,-64.157,73.95,76.137684,1981,12
36631,-63.762,75.25,48.932214,1981,12
38003,-63.629,75.55,44.453259,1981,12
...,...,...,...,...,...
18988,-64.633,148.75,108.840803,2013,12
20857,-64.547,148.85,101.984896,2013,12
34771,-63.938,149.15,48.454271,2013,12


## Joining masked data frame with background data frame
We will use the grid cell coordinates and dates to perform this join.

In [76]:
crabeaters = crabeaters.merge(var_pd, on = cols, how = 'left')
crabeaters = crabeaters.drop_duplicates()
crabeaters

,date,year,month,longitude,latitude,xt_ocean,yt_ocean,season_year,life_stage,decade,...,zone,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m,SIC,SST_degC,lt_pack_ice,dist_ice_edge_km
0,1999-12,1999,12,75.05,-69.25,75.05,-69.239,summer,weaning,1990,...,Antarctic,0,89.812,504.080,140.256,777.867,NaN,-1.506463,0.000000,724.237926
1,1999-12,1999,12,75.85,-69.25,75.85,-69.239,summer,weaning,1990,...,Antarctic,0,89.953,506.158,143.956,503.000,0.955706,-1.604321,0.535714,708.715428
2,1999-12,1999,12,76.05,-69.25,76.05,-69.239,summer,weaning,1990,...,Antarctic,0,89.949,507.046,145.997,522.900,0.945087,-1.573719,0.547619,705.058053
3,1999-12,1999,12,77.25,-69.25,77.25,-69.239,summer,weaning,1990,...,Antarctic,0,NaN,509.341,166.208,NaN,NaN,NaN,0.000000,685.100828
4,1999-12,1999,12,73.95,-69.15,73.95,-69.155,summer,weaning,1990,...,Antarctic,0,NaN,494.995,130.925,599.817,NaN,-1.313767,0.000000,740.003921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46987,1999-12,1999,12,69.25,-59.45,69.25,-59.442,summer,weaning,1990,...,Antarctic,0,89.833,556.221,688.617,4624.497,0.000000,-0.626995,0.000000,289.197201
46988,1999-12,1999,12,71.95,-59.45,71.95,-59.442,summer,weaning,1990,...,Antarctic,0,89.653,536.648,699.380,4429.308,0.000000,-0.988253,0.035714,276.617004
46989,1987-11,1987,11,70.85,-59.35,70.85,-59.340,autumn,weaning,1980,...,Antarctic,0,89.980,551.535,703.061,4767.584,0.044812,-1.306596,0.000000,28.266072
46990,1998-11,1998,11,71.55,-59.35,71.55,-59.340,autumn,weaning,1990,...,Antarctic,0,89.930,548.571,707.330,4608.486,0.038312,-1.382793,0.035714,32.938866


## Saving data frame to disk

In [77]:
#Ensure output folder exists
os.makedirs(varDict['base_out'], exist_ok = True)

#Create file path where data will be saved
file_out = os.path.join(varDict['base_out'], 'unique_background_20x_obs_all_env.csv')

#Saving as csv file
crabeaters.to_csv(file_out, index = False)